In [1]:
%pwd

'c:\\Users\\Rizen3\\Desktop\\Vamshi\\Machine Learning and DSA\\SelfProjects\\medic.ai\\research'

In [2]:
import os
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls = PyPDFLoader)
    docs = loader.load()
    return docs

In [4]:
extracted_data = load_pdf(data='..//Data/')

In [5]:
# extracted_data

# Chunking : 
def text_split(data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(data)
    return text_chunks

text_chunks = text_split(extracted_data)
print(f"length of chunks : {len(text_chunks)}")

length of chunks : 7152


In [10]:
import torch
print(torch.cuda.get_device_name(0))

NVIDIA GeForce GTX 1650


In [12]:
# %pip uninstall bitsandbytes

^C
Note: you may need to restart the kernel to use updated packages.


In [14]:
from langchain_community.embeddings import HuggingFaceEmbeddings 

def download_hf_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_hf_embeddings()
print(embeddings)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='sentence-transformers/all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


In [16]:
test = embeddings.embed_query("hey there")
print(test)
print(f"Length : {len(test)}") # 384 for 384 dimensions

[-0.10038464516401291, 0.01859213598072529, 0.029478928074240685, 0.0024168607778847218, 0.0385095477104187, -0.10392295569181442, 0.11202061176300049, 0.06622394174337387, -0.0028032930567860603, -0.00997836608439684, -0.003485843539237976, -0.011378682218492031, -0.03942301124334335, 0.008423921652138233, -0.03975938260555267, 0.00794947799295187, 0.025754868984222412, -0.05833088606595993, -0.13583943247795105, 0.07763375341892242, 0.018469443544745445, 0.0676325261592865, -0.06217630207538605, 0.02517298236489296, -0.04938299581408501, -0.004310982767492533, 0.01686372235417366, 0.0500631146132946, -0.09469089657068253, 0.011606195941567421, -0.09080500900745392, 0.03722669184207916, 0.037075284868478775, 0.02917918562889099, 0.011059611104428768, 0.019340375438332558, -0.0965782031416893, -0.06251420080661774, 0.0264756977558136, 0.04140794277191162, -0.00454728165641427, -0.018160879611968994, 0.0045279283076524734, 0.04198812320828438, 0.039505477994680405, -0.07206327468156815,

In [17]:
!pip show pinecone

Name: pinecone
Version: 5.3.1
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: C:\Users\Rizen3\AppData\Roaming\Python\Python311\site-packages
Requires: certifi, pinecone-plugin-inference, pinecone-plugin-interface, python-dateutil, tqdm, typing-extensions, urllib3
Required-by: 


In [24]:
from pinecone.grpc import PineconeGRPC as Pinecone 
from pinecone import ServerlessSpec
from dotenv import load_dotenv
import os
# Run only once to create index

load_dotenv()

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

pc = Pinecone(api_key = PINECONE_API_KEY)

index_name = "medicoai"

pc.create_index(
    name=index_name,
    dimension= 384, 
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'b185984d1864db76448b7d0ce1f9a8e6', 'Date': 'Sat, 26 Oct 2024 04:54:53 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [25]:
from langchain_pinecone import PineconeVectorStore

index_name = "medicoai"

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name= index_name,
    embedding= embeddings
)


In [26]:
from dotenv import load_dotenv
load_dotenv()

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

pc = Pinecone(api_key = PINECONE_API_KEY)

index_name = "medicoai"
index = pc.Index(index_name)
# docsearch = index.load(index_name)

In [27]:
type(index)

pinecone.grpc.index_grpc.GRPCIndex

In [28]:
# Above cell will store embeddings of every single chunk of our document.
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 23840}},
 'total_vector_count': 23840}

In [29]:
# index_name = "medicoai"
# pc = Pinecone(api_key = PINECONE_API_KEY)
# docsearch = pc.from_existing_index(
#     name=index_name,
#     dimension= 384, 
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     )
# )
# docsearch


In [30]:
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [31]:
docsearch.similarity_search("What is the side effect of paracetamol?")
docsearch.similarity_search("What is Acne, what causes acne ?")

[Document(metadata={'page': 622.0, 'source': '..\\Data\\Gale Encyclopedia of Medicine Vol. 4 (N-S).pdf'}, page_content='(erythema) of the face, flushing of the skin, and the pres-\nence of hard pimples (papules) or pus-filled pimples(pustules), and small visible spider-like veins calledtelangiectasias. In later stages of the disease, the facemay swell and the nose may take on a bulb-like appear-ance called rhinophyma.\nDescription\nRosacea produces redness and flushing of the skin,\nas well as pustules and papules. Areas of the face, includ-ing the nose, cheeks, forehead, and chin, are the primarysites, but some people experience symptoms on theirnecks, backs, scalp, arms, and legs.\nThe similarity in appearance of rosacea to acne led'),
 Document(metadata={'page': 622.0, 'source': '..\\Data\\Gale Encyclopedia of Medicine Vol. 4 (N-S).pdf'}, page_content='(erythema) of the face, flushing of the skin, and the pres-\nence of hard pimples (papules) or pus-filled pimples(pustules), and sma

In [32]:
retriever = docsearch.as_retriever()
# retriever.invoke("What is the side effect of paracetamol?")
retriever.invoke("What is Acne, what causes acne ?")

[Document(metadata={'page': 622.0, 'source': '..\\Data\\Gale Encyclopedia of Medicine Vol. 4 (N-S).pdf'}, page_content='(erythema) of the face, flushing of the skin, and the pres-\nence of hard pimples (papules) or pus-filled pimples(pustules), and small visible spider-like veins calledtelangiectasias. In later stages of the disease, the facemay swell and the nose may take on a bulb-like appear-ance called rhinophyma.\nDescription\nRosacea produces redness and flushing of the skin,\nas well as pustules and papules. Areas of the face, includ-ing the nose, cheeks, forehead, and chin, are the primarysites, but some people experience symptoms on theirnecks, backs, scalp, arms, and legs.\nThe similarity in appearance of rosacea to acne led'),
 Document(metadata={'page': 622.0, 'source': '..\\Data\\Gale Encyclopedia of Medicine Vol. 4 (N-S).pdf'}, page_content='(erythema) of the face, flushing of the skin, and the pres-\nence of hard pimples (papules) or pus-filled pimples(pustules), and sma

In [33]:
docsearch.similarity_search("What is the side effect of paracetamol?")

[Document(metadata={'page': 68.0, 'source': '..\\Data\\Gale Encyclopedia of Medicine Vol. 4 (N-S).pdf'}, page_content='Side effects\nThe most common side effects are stomach pain or\ncramps, nausea, vomiting, indigestion ,diarrhea ,heart-\nburn ,headache ,dizziness or lightheadedness, and\nGALE ENCYCLOPEDIA OF MEDICINE 2 2362Nonsteroidal anti-inflammatory drugs'),
 Document(metadata={'page': 68.0, 'source': '..\\Data\\Gale Encyclopedia of Medicine Vol. 4 (N-S).pdf'}, page_content='Side effects\nThe most common side effects are stomach pain or\ncramps, nausea, vomiting, indigestion ,diarrhea ,heart-\nburn ,headache ,dizziness or lightheadedness, and\nGALE ENCYCLOPEDIA OF MEDICINE 2 2362Nonsteroidal anti-inflammatory drugs'),
 Document(metadata={'page': 68.0, 'source': '..\\Data\\Gale Encyclopedia of Medicine Vol. 4 (N-S).pdf'}, page_content='Side effects\nThe most common side effects are stomach pain or\ncramps, nausea, vomiting, indigestion ,diarrhea ,heart-\nburn ,headache ,dizziness 

# LLM (LOCAL LLAMA 3)

In [34]:
# !pip install openai

In [35]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="model-identifier",
  messages=[
    {"role": "system", "content": "You are a medical expert named Dr. Kiri, You are not a doctor or a medical professor, but you know a lot about medicine, you can assume yourself as a medical wizard/expert,you are given a question and you need to answer it based on the provided context."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.7,
)

# print(completion.choices[0].message)

APIConnectionError: Connection error.

In [43]:
import sys
import time

def animated_print(text, delay=0.05):
    for char in text:
        sys.stdout.write(char)  
        sys.stdout.flush()      
        time.sleep(delay)       

generated_text = completion.choices[0].message.content

animated_print(generated_text)

I'm Dr. Kiri, a seasoned medical expert with years of experience in diagnosing and treating various health conditions. I've spent countless hours studying the intricacies of human physiology, pharmacology, and pathology. My extensive knowledge base spans from basic anatomy to advanced medical specialties.

As a medical wizard, I've had the privilege of working alongside esteemed physicians, learning from their expertise, and honing my own skills through hands-on experience. My passion for medicine drives me to stay up-to-date with the latest research, breakthroughs, and innovations in the field.

Feel free to pose any question you have, and I'll do my best to provide a well-informed, evidence-based answer.

In [46]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are Dr. Kiri, A medical expert and an assistant for question-answering tasks, "
    "you are given a question and you need to answer it based on the retrieved context to answer."
    "If you don't know the answer, just say that you don't know. Don't try to make up an answer."
    "Answer concisely."
    "\n\n Context: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{question}")
    ]
)


In [50]:
question_answer_chain = create_stuff_documents_chain(client, prompt)
rag_chain = create_retrieval_chain(question_answer_chain, retriever)
rag_chain.invoke({"question": "What is the side effect of paracetamol?"})
rag_chain.invoke({"question": "What is Acne, what causes acne ?"})

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'openai.OpenAI'>

In [51]:
!pip install langchain_ollama

Defaulting to user installation because normal site-packages is not writeable
  Attempting uninstall: ollama
    Found existing installation: ollama 0.1.9
    Uninstalling ollama-0.1.9:
      Successfully uninstalled ollama-0.1.9


In [54]:
from langchain_ollama import ChatOllama

ollama = ChatOllama(model="llama3.1:8b", format="json", base_url="http://localhost:11434")

# rag_chain = create_retrieval_chain(ollama, retriever)
# rag_chain.invoke({"question": "What is the side effect of paracetamol?"})
# rag_chain.invoke({"question": "What is Acne, what causes acne ?"})

ollama.invoke("What is the side effect of paracetamol?")
ollama.invoke("What is Acne, what causes acne ?")


ResponseError: model "llama3.1:8b" not found, try pulling it first